In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import keras

Using TensorFlow backend.


In [2]:
input_type = 'train'
agg_roles_dict={'Role':  lambda x : x.mode() }
df_rol_raw = pd.read_csv('../data/' + input_type + '_roles_raw.csv', low_memory=False)
df_rol_raw.head()

,id,Person.ID,Role
0,1,40572,CHIEF_INVESTIGATOR
1,2,9067,CHIEF_INVESTIGATOR
2,3,5967,CHIEF_INVESTIGATOR
3,4,78782,PRINCIPAL_SUPERVISOR
4,5,13042,CHIEF_INVESTIGATOR


In [48]:
# we generate a dataframe with the uniqe key pairs ( date, person Id).
# this df will be filled and returned as the mod frame
df_rol_mod = df_rol_raw.loc[:,['id', 'Person.ID']].drop_duplicates()


for col, fun in agg_roles_dict.items():
 
    # we apply the aggregation function to the coloumn
    tmp2 = df_rol_raw.groupby(['id', 'Person.ID'])[col].agg(fun)
    
    # in case the aggregation function returns a list and not a single element, we take the first one
    # if there were only NaN for this day and persion ID, an empty np array is returned-> we chagne it to a NaN
    tmp2 = tmp2.apply(lambda x : x[0] if (isinstance(x, np.ndarray) and len(x) > 0 ) else \
               (np.nan if (isinstance(x, np.ndarray) and len(x) == 0) else x  ))
    
    # we merge the series with the dataframe that stores all the outcome
    df_rol_mod = pd.merge(df_rol_mod, tmp2.to_frame(name = col).reset_index(),how='left',on = ['id', 'Person.ID'])

In [31]:
# read all csv data
df_roles = pd.read_csv('../data/' + input_type + '_role_mod.csv', low_memory=False)
df_p_dyn = pd.read_csv('../data/' + input_type + '_person_dyn_mod.csv', low_memory=False,parse_dates=['date'])
df_p_static = pd.read_csv('../data/' + input_type + '_person_static_mod.csv', low_memory=False)
#df_ext = pd.read_csv('../data/' + input_type + '_externals_raw.csv', low_memory=False)

# we also need the combination (id,date)
df_dates = pd.read_csv('../data/' + input_type + '.csv', low_memory=False,parse_dates=['date']).loc[:, ['id','date']].drop_duplicates()

# we also need the combination (id,date)
df_main = pd.read_csv('../data/' + input_type + '.csv', low_memory=False,parse_dates=['date'])


In [33]:
df_main.head(15)

,id,granted,sponsor,grant_category,grant_value,date,RFCD.Code.1,RFCD.Percentage.1,RFCD.Code.2,RFCD.Percentage.2,RFCD.Code.3,RFCD.Percentage.3,RFCD.Code.4,RFCD.Percentage.4,RFCD.Code.5,RFCD.Percentage.5,SEO.Code.1,SEO.Percentage.1,SEO.Code.2,SEO.Percentage.2,SEO.Code.3,SEO.Percentage.3,SEO.Code.4,SEO.Percentage.4,SEO.Code.5,SEO.Percentage.5,Person.ID.1,Role.1,Year.of.Birth.1,Country.of.Birth.1,Home.Language.1,Dept.No..1,Faculty.No..1,With.PHD.1,No..of.Years.in.Uni.at.Time.of.Grant.1,Number.of.Successful.Grant.1,Number.of.Unsuccessful.Grant.1,A..1,A.1,B.1,C.1,Person.ID.2,Role.2,Year.of.Birth.2,Country.of.Birth.2,Home.Language.2,Dept.No..2,Faculty.No..2,With.PHD.2,No..of.Years.in.Uni.at.Time.of.Grant.2,Number.of.Successful.Grant.2,Number.of.Unsuccessful.Grant.2,A..2,A.2,B.2,C.2,Person.ID.3,Role.3,Year.of.Birth.3,Country.of.Birth.3,Home.Language.3,Dept.No..3,Faculty.No..3,With.PHD.3,No..of.Years.in.Uni.at.Time.of.Grant.3,Number.of.Successful.Grant.3,Number.of.Unsuccessful.Grant.3,A..3,A.3,B.3,C.3,Person.ID.4,Role.4,Year.of.Birth.4,Country.of.Birth.4,Home.Language.4,Dept.No..4,Faculty.No..4,With.PHD.4,No..of.Years.in.Uni.at.Time.of.Grant.4,Number.of.Successful.Grant.4,Number.of.Unsuccessful.Grant.4,A..4,A.4,B.4,C.4,Person.ID.5,Role.5,Year.of.Birth.5,Country.of.Birth.5,Home.Language.5,Dept.No..5,Faculty.No..5,With.PHD.5,No..of.Years.in.Uni.at.Time.of.Grant.5,Number.of.Successful.Grant.5,Number.of.Unsuccessful.Grant.5,A..5,A.5,B.5,C.5,Person.ID.6,Role.6,Year.of.Birth.6,Country.of.Birth.6,Home.Language.6,Dept.No..6,Faculty.No..6,With.PHD.6,No..of.Years.in.Uni.at.Time.of.Grant.6,Number.of.Successful.Grant.6,Number.of.Unsuccessful.Grant.6,A..6,A.6,B.6,C.6,Person.ID.7,Role.7,Year.of.Birth.7,Country.of.Birth.7,Home.Language.7,Dept.No..7,Faculty.No..7,With.PHD.7,No..of.Years.in.Uni.at.Time.of.Grant.7,Number.of.Successful.Grant.7,Number.of.Unsuccessful.Grant.7,A..7,A.7,B.7,C.7,Person.ID.8,Role.8,Year.of.Birth.8,Country.of.Birth.8,Home.Language.8,Dept.No..8,Faculty.No..8,With.PHD.8,No..of.Years.in.Uni.at.Time.of.Grant.8,Number.of.Successful.Grant.8,Number.of.Unsuccessful.Grant.8,A..8,A.8,B.8,C.8,Person.ID.9,Role.9,Year.of.Birth.9,Country.of.Birth.9,Home.Language.9,Dept.No..9,Faculty.No..9,With.PHD.9,No..of.Years.in.Uni.at.Time.of.Grant.9,Number.of.Successful.Grant.9,Number.of.Unsuccessful.Grant.9,A..9,A.9,B.9,C.9,Person.ID.10,Role.10,Year.of.Birth.10,Country.of.Birth.10,Home.Language.10,Dept.No..10,Faculty.No..10,With.PHD.10,No..of.Years.in.Uni.at.Time.of.Grant.10,Number.of.Successful.Grant.10,Number.of.Unsuccessful.Grant.10,A..10,A.10,B.10,C.10,Person.ID.11,Role.11,Year.of.Birth.11,Country.of.Birth.11,Home.Language.11,Dept.No..11,Faculty.No..11,With.PHD.11,No..of.Years.in.Uni.at.Time.of.Grant.11,Number.of.Successful.Grant.11,Number.of.Unsuccessful.Grant.11,A..11,A.11,B.11,C.11,Person.ID.12,Role.12,Year.of.Birth.12,Country.of.Birth.12,Home.Language.12,Dept.No..12,Faculty.No..12,With.PHD.12,No..of.Years.in.Uni.at.Time.of.Grant.12,Number.of.Successful.Grant.12,Number.of.Unsuccessful.Grant.12,A..12,A.12,B.12,C.12,Person.ID.13,Role.13,Year.of.Birth.13,Country.of.Birth.13,Home.Language.13,Dept.No..13,Faculty.No..13,With.PHD.13,No..of.Years.in.Uni.at.Time.of.Grant.13,Number.of.Successful.Grant.13,Number.of.Unsuccessful.Grant.13,A..13,A.13,B.13,C.13,Person.ID.14,Role.14,Year.of.Birth.14,Country.of.Birth.14,Home.Language.14,Dept.No..14,Faculty.No..14,With.PHD.14,No..of.Years.in.Uni.at.Time.of.Grant.14,Number.of.Successful.Grant.14,Number.of.Unsuccessful.Grant.14,A..14,A.14,B.14,C.14,Person.ID.15,Role.15,Year.of.Birth.15,Country.of.Birth.15,Home.Language.15,Dept.No..15,Faculty.No..15,With.PHD.15,No..of.Years.in.Uni.at.Time.of.Grant.15,Number.of.Successful.Grant.15,Number.of.Unsuccessful.Grant.15,A..15,A.15,B.15,C.15
0,1,1,NaN,NaN,A,2005-08-11,280199.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,700299.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40572.0,CHIEF_INVESTIGATOR,1965.0,Asia Pacific,Other,3073.0,31.0,NaN,Less than 0,0.0,0.0,4.0,2.0,0.0,0.0,NaN,NaN,NaN,

In [67]:
rfcd_level = 1
rfcd_level_dict={0:10000,
          1:100,
          2:1}
apps_rfcd = df_main.set_index('id').filter(regex = '^RFCD.Code.').\
    reset_index().melt('id').query("value != 0.").\
    assign(rfcd = lambda df : df.value // rfcd_level_dict[rfcd_level] ).sort_values('id').\
    reset_index(drop = True).loc[:,['id','rfcd']].drop_duplicates()

In [68]:
apps_rfcd

,id,rfcd
0,1,2801.0
1,2,2802.0
2,2,2801.0
4,3,3212.0
5,3,3210.0
6,4,2706.0
7,4,3206.0
8,5,2800.0
9,5,2605.0
10,5,2900.0


In [136]:
rfcd_level = 1
rfcd_level_dict={0:10000,
          1:100,
          2:1}
apps_rfcd = df_main.set_index('id').filter(regex = '^RFCD.Code.').\
    reset_index().melt('id').query("value != 0.").\
    assign(rfcd = lambda df : df.value // rfcd_level_dict[rfcd_level] ).sort_values('id').\
    reset_index(drop = True).loc[:,['id','rfcd']].drop_duplicates()

#the df filter stores the (id, filter.value)
df_filter = apps_rfcd

df_filter.columns = ['id','f_value']

result = pd.DataFrame(columns=['s_count','s_rate'])

for id in apps_rfcd.id.unique():
    #print(id)
    team = list(df_roles.loc[df_roles.id == id,'Person.ID'])
    
    #this is a list of all applications, also future ones
    all_team_apps = pd.Series(df_roles.loc[df_roles.loc[:,'Person.ID'].isin(team),'id'].unique())
    
    # we now need to filter for previous applications of the team 
    date_this_app = df_dates.loc[df_dates.id == id,'date']
    ids_all_prev_apps = list(df_dates.loc[ df_dates.date < date_this_app.iloc[0],'id'])
    prev_team_apps = list(all_team_apps[all_team_apps.isin(ids_all_prev_apps)])
    
    # of those previous applications, we now keep only the ones that were in the same category
    # ie same sponsor, same rfcd
    current_filter = list(df_filter.loc[df_filter.id  == id,'f_value'].unique())
    prev_team_apps_filter = list(df_filter.\
                                 loc[df_filter.f_value.isin(current_filter) & df_filter.id.isin(prev_team_apps),'id' ])
    
    granted_filter = df_main.loc[df_main.id.isin(prev_team_apps_filter),'granted']
    s_count = granted_filter.sum()
    s_rate = granted_filter.mean()
    
    tmp=pd.DataFrame({'s_count' :  granted_filter.sum(),
                     's_rate' : granted_filter.mean()},index=[id])
    result=result.append(tmp)  

In [170]:
def gen_s(df_filter, df_roles,df_main):
    df_dates = df_main.loc[:, ['id','date']].drop_duplicates()
    df_filter.columns = ['id', 'f_value']

    result = pd.DataFrame(columns=['s_count', 's_rate'])

    for id in df_filter.id.unique():
        # print(id)
        team = list(df_roles.loc[df_roles.id == id, 'Person.ID'])

        # this is a list of all applications, also future ones
        all_team_apps = pd.Series(df_roles.loc[df_roles.loc[:, 'Person.ID'].isin(team), 'id'].unique())

        # we now need to filter for previous applications of the team
        date_this_app = df_dates.loc[df_dates.id == id, 'date']
        ids_all_prev_apps = list(df_dates.loc[df_dates.date < date_this_app.iloc[0], 'id'])
        prev_team_apps = list(all_team_apps[all_team_apps.isin(ids_all_prev_apps)])

        # of those previous applications, we now keep only the ones that were in the same category
        # ie same sponsor, same rfcd
        current_filter = list(df_filter.loc[df_filter.id == id, 'f_value'].unique())
        prev_team_apps_filter = list(df_filter. \
                                     loc[df_filter.f_value.isin(current_filter) & df_filter.id.isin(prev_team_apps), 'id'])

        granted_filter = df_main.loc[df_main.id.isin(prev_team_apps_filter), 'granted']

        tmp = pd.DataFrame({'s_count': granted_filter.sum(),
                            's_rate': granted_filter.mean()}, index=[id])
        result = result.append(tmp)
    
    result.index.names = ['id']
    result = result.reset_index()
    return result

def gen_filter_rfcd(df_main, rfcd_level = 1):
    rfcd_level_dict = {0: 10000,
                       1: 100,
                       2: 1}
    apps_rfcd = df_main.set_index('id').filter(regex='^RFCD.Code.'). \
                    reset_index().melt('id').query("value != 0."). \
                    assign(rfcd=lambda df: df.value // rfcd_level_dict[rfcd_level]).sort_values('id'). \
                    reset_index(drop=True).loc[:, ['id', 'rfcd']].drop_duplicates()

    return apps_rfcd

In [179]:
df_out = df_main.loc[:,['id']]
for j in range(3):
    print('rfcd ' + str(j))
    df_filter = gen_filter_rfcd(df_main,j)
    tmp_res = gen_s(df_filter, df_roles, df_main)
    tmp_res.columns =  ['rfcd_l' + str(j) + '_' + x  if x !='id' else x  for x in tmp_res.columns]
    df_out = df_out.merge(tmp_res,how = 'left', on = 'id')
   
    
#gen_filter_rfcd

rfcd 0
rfcd 1
rfcd 2


In [183]:
col = 'sponsor'
df_filter = df_main.set_index('id').filter(regex='^' + col). \
                    reset_index().melt('id').query("value != 0.").\
                    sort_values('id'). \
                    reset_index(drop=True).loc[:, ['id', 'value']].drop_duplicates()
df_filter   

,id,value
0,1,NaN
1,2,2B
2,3,29A
3,4,40D
4,5,59C
5,6,4D
6,7,2B
7,8,28D
8,9,2B
9,10,2B


In [178]:
tmp_res.tail(50)
tmp = tmp_res.copy()


tmp.columns = ['rfcd_l' + str(j) + '_' + x  if x !='id' else x  for x in tmp_res.columns]
#tmp

In [162]:
feature_dict = {'Role' : {'n_internals' :lambda x : x.size},
                'Year.of.Birth': { 'mean_year_of_birth' : lambda x : x.mean(),
                                 'max_year_of_birth' : lambda x : x.max()},
            'Country.of.Birth': {'Country.of.Birth': lambda x : x.mode()},
           'Home.Language':  {'Home.Language': lambda x : x.mode()},
           'Dept.No.' :  {'Dept.No.' : lambda x : x.mode()},
             'Faculty.No.':  {'Faculty.No.': lambda x : x.mode()},
              'With.PHD' : {'With.PHD' : lambda x : x.sum()},
               'years_in_uni' : {'mean_years_in_uni' : lambda x : x.mean(),
                                'max_years_in_uni' : lambda x : x.max()},
               'Number.of.Successful.Grant' : {'Number.of.Successful.Grant' : lambda x : x.sum()},
               'Number.of.Unsuccessful.Grant': {'Number.of.Unsuccessful.Grant' : lambda x : x.sum()},
                'A.':{'A.':lambda x : x.sum()},
               'A': {'A': lambda x : x.sum()},
               'B': {'B':lambda x : x.sum()},
                'C':{'C': lambda x : x.sum()}}

In [32]:
persons = df_dates.merge(df_roles, how = 'left', on = ['id']).\
    merge(df_p_static,how = 'left', on = ['Person.ID']).\
    merge(df_p_dyn,how = 'left', on = ['date','Person.ID'])

In [95]:
persons

,id,date,Person.ID,Role,Year.of.Birth,Country.of.Birth,Home.Language,Dept.No.,Faculty.No.,With.PHD,years_in_uni,Number.of.Successful.Grant,Number.of.Unsuccessful.Grant,A.,A,B,C
0,1,2005-08-11,40572,CHIEF_INVESTIGATOR,1965.0,Asia Pacific,Other,3073.0,31.0,NaN,0.0,0.0,0.0,4.0,2.0,0.0,0.0
1,2,2005-11-11,9067,CHIEF_INVESTIGATOR,1960.0,Australia,English,2538.0,25.0,1.0,4.0,0.0,0.0,6.0,12.0,2.0,2.0
2,3,2005-11-14,5967,CHIEF_INVESTIGATOR,1955.0,Australia,English,2923.0,25.0,1.0,2.0,0.0,0.0,0.0,3.0,5.0,2.0
3,3,2005-11-14,27307,CHIEF_INVESTIGATOR,1950.0,Australia,English,2923.0,25.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3,2005-11-14,79652,CHIEF_INVESTIGATOR,1950.0,Asia Pacific,Other,2498.0,25.0,1.0,4.0,0.0,0.0,1.0,3.0,3.0,3.0
5,3,2005-11-14,11667,DELEGATED_RESEARCHER,1950.0,Australia,English,2548.0,25.0,NaN,4.0,0.0,0.0,6.0,14.0,12.0,2.0
6,4,2005-11-15,78782,PRINCIPAL_SUPERVISOR,1955.0,Australia,English,2678.0,25.0,1.0,2.0,0.0,0.0,0.0,3.0,13.0,3.0
7,4,2005-11-15,55337,CHIEF_INVESTIGATOR,1975.0,Australia,English,2678.0,25.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8,5,2005-11-16,13042,CHIEF_INVESTIGATOR,1965.0,Western Europe,Other,2153.0,19.0,1.0,2.0,0.0,0.0,3.0,0.0,1.0,0.0
9,6,2005-11-19,301472,CHIEF_INVESTIGATOR,1950.0,Australia,English,2533.0,25.0,1.0,1.0,2.0,0.0,7.0,27.0,27.0,6.0


In [99]:
# we generate a dataframe with the uniqe key pairs ( date, person Id).
# this df will be filled and returned as the mod frame
df_out = df_dates.loc[:,['id']].drop_duplicates()


for col_in, fun_dict in feature_dict.items():
    for col_out, fun in fun_dict.items():
        print(col_out)
 
        # we apply the aggregation function to the coloumn
        tmp2 = persons.groupby(['id'])[col_in].agg(fun)
    
        # in case the aggregation function returns a list and not a single element, we take the first one
        # if there were only NaN for this day and persion ID, an empty np array is returned-> we chagne it to a NaN
        tmp2 = tmp2.apply(lambda x : x[0] if (isinstance(x, np.ndarray) and len(x) > 0 ) else \
               (np.nan if (isinstance(x, np.ndarray) and len(x) == 0) else x  ))
    
        # we merge the series with the dataframe that stores all the outcome
        df_out = pd.merge(df_out, tmp2.to_frame(name = col_out).reset_index(),how='left',on = ['id'])

Dept.No.
Faculty.No.
A.
A
C
Number.of.Successful.Grant
Country.of.Birth
Home.Language
Number.of.Unsuccessful.Grant
max_year_of_birth
mean_year_of_birth
mean_years_in_uni
max_years_in_uni
n_internals
With.PHD
B


In [100]:
df_out

,id,Dept.No.,Faculty.No.,A.,A,C,Number.of.Successful.Grant,Country.of.Birth,Home.Language,Number.of.Unsuccessful.Grant,max_year_of_birth,mean_year_of_birth,mean_years_in_uni,max_years_in_uni,n_internals,With.PHD,B
0,1,3073.0,31.0,4.0,2.0,0.0,0.0,Asia Pacific,Other,0.0,1965.0,1965.000000,0.000000,0.0,1,NaN,0.0
1,2,2538.0,25.0,6.0,12.0,2.0,0.0,Australia,English,0.0,1960.0,1960.000000,4.000000,4.0,1,1.0,2.0
2,3,2923.0,25.0,7.0,20.0,7.0,0.0,Australia,English,0.0,1955.0,1951.250000,2.500000,4.0,4,2.0,20.0
3,4,2678.0,25.0,0.0,3.0,3.0,0.0,Australia,English,0.0,1975.0,1965.000000,1.500000,2.0,2,2.0,13.0
4,5,2153.0,19.0,3.0,0.0,0.0,0.0,Western Europe,Other,0.0,1965.0,1965.000000,2.000000,2.0,1,1.0,1.0
5,6,2533.0,25.0,7.0,27.0,6.0,2.0,Australia,English,0.0,1950.0,1950.000000,1.000000,1.0,1,1.0,27.0
6,7,3028.0,31.0,9.0,15.0,5.0,0.0,Australia,English,1.0,1960.0,1956.666667,2.000000,3.0,3,2.0,20.0
7,8,2563.0,25.0,0.0,0.0,0.0,0.0,Australia,English,3.0,1965.0,1952.500000,0.000000,0.0,2,NaN,0.0
8,9,3123.0,31.0,14.0,1.0,0.0,2.0,Australia,English,1.0,1950.0,1947.500000,2.000000,2.0,2,2.0,16.0
9,10,NaN,NaN,1.0,0.0,0.0,0.0,Unknown,Unknown,0.0,1930.0,1930.000000,NaN,NaN,1,NaN,0.0


In [51]:
df_rol_raw.loc[:,['id', 'Person.ID']].drop_duplicates().shape

(9735, 2)

In [19]:
persons = pd.concat( [df.iloc[:,0],  df.loc[:,'Person.ID.1':'C.15']], axis = 1)

In [153]:
persons.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6633 entries, 0 to 6632
Columns: 226 entries, id to C.15
dtypes: float64(159), int64(1), object(66)
memory usage: 11.4+ MB


In [20]:
persons.head()

,id,Person.ID.1,Role.1,Year.of.Birth.1,Country.of.Birth.1,Home.Language.1,Dept.No..1,Faculty.No..1,With.PHD.1,No..of.Years.in.Uni.at.Time.of.Grant.1,...,Dept.No..15,Faculty.No..15,With.PHD.15,No..of.Years.in.Uni.at.Time.of.Grant.15,Number.of.Successful.Grant.15,Number.of.Unsuccessful.Grant.15,A..15,A.15,B.15,C.15
0,1,40572.0,CHIEF_INVESTIGATOR,1965.0,Asia Pacific,Other,3073.0,31.0,NaN,Less than 0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,9067.0,CHIEF_INVESTIGATOR,1960.0,Australia,NaN,2538.0,25.0,Yes,more than 15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,5967.0,CHIEF_INVESTIGATOR,1955.0,Australia,NaN,2923.0,25.0,Yes,>5 to 10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,78782.0,PRINCIPAL_SUPERVISOR,1955.0,Australia,NaN,2678.0,25.0,Yes,>5 to 10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,13042.0,CHIEF_INVESTIGATOR,1965.0,Western Europe,NaN,2153.0,19.0,Yes,>5 to 10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
#we flatten the persons subtable
#all columns with persons informatin is repeated 15 times
n_persons = 15

#this gives us the following number of coloumns per person (also 15):
n_persons_cols = (len(persons.columns) - 1) // n_persons

#we generate a list the with ID and the field names
col_names = [persons.columns[0]]
[col_names.append(i[:-2]) for i in persons.columns[1:n_persons_cols+1]]

# and use this list to make an empty dataframe
df_p = pd.DataFrame(columns = col_names)

#we loop over all persons 1..15 and take the respective fields (ie columns) and append them
for i in range(n_persons):
    #coloumn index for first coloumn of person i
    k = 1 + (i*n_persons_cols)
    #coloumn index for last coloumn of person i
    l = 1 + ((i+1)*n_persons_cols)
    #get a temp table with the desired info
    tmp = pd.concat( [persons.iloc[:,0],  persons.iloc[:,k:l]], axis = 1)
    #rename the table to make the UNION ALL opertion work
    tmp.columns = df_p.columns
    df_p = pd.concat([df_p, tmp],axis = 0)



In [22]:
# remove lines with person ID AND Role empty
df_p = df_p.loc[~(df_p.loc[:,'Person.ID'].isnull() & df_p.loc[:,'Role'].isnull()),:]

In [23]:
df_p.head()

,id,Person.ID,Role,Year.of.Birth,Country.of.Birth,Home.Language,Dept.No.,Faculty.No.,With.PHD,No..of.Years.in.Uni.at.Time.of.Grant,Number.of.Successful.Grant,Number.of.Unsuccessful.Grant,A.,A,B,C
0,1,40572.0,CHIEF_INVESTIGATOR,1965.0,Asia Pacific,Other,3073.0,31.0,NaN,Less than 0,0.0,0.0,4.0,2.0,0.0,0.0
1,2,9067.0,CHIEF_INVESTIGATOR,1960.0,Australia,NaN,2538.0,25.0,Yes,more than 15,0.0,0.0,6.0,12.0,2.0,2.0
2,3,5967.0,CHIEF_INVESTIGATOR,1955.0,Australia,NaN,2923.0,25.0,Yes,>5 to 10,0.0,0.0,0.0,3.0,5.0,2.0
3,4,78782.0,PRINCIPAL_SUPERVISOR,1955.0,Australia,NaN,2678.0,25.0,Yes,>5 to 10,0.0,0.0,0.0,3.0,13.0,3.0
4,5,13042.0,CHIEF_INVESTIGATOR,1965.0,Western Europe,NaN,2153.0,19.0,Yes,>5 to 10,0.0,0.0,3.0,0.0,1.0,0.0


In [24]:
# we create a new table called externals with the  columns grant_application_id and has_[external role]
# external roles are stored as person without ID, therefore the following command identifies them:
# df_p.loc[df_p.loc[:,'Person.ID'].isnull(),'Role'].unique()
# of course they can also be identifed by name (ie external advisor..)
# it might be possilbe that a different role has a missing person ID filed in the testing date,
# therefore we hardcode it: EXT_CHIEF_INVESTIGATOR', 'STUD_CHIEF_INVESTIGATOR', 'STUDRES', 'EXTERNAL_ADVISOR'
ext_list = ['EXT_CHIEF_INVESTIGATOR', 'STUD_CHIEF_INVESTIGATOR', 'STUDRES', 'EXTERNAL_ADVISOR']
mask = df_p.Role.isin(ext_list)
df_ext = df_p.loc[mask,:]
#we only want to keep the grant application id and the role
df_ext = df_ext.loc[:,['id','Role']]

df_p = df_p.loc[~mask,:]

In [25]:
# next step is to pivot the table so that we have one line per grant applications ID
df_ext['dummy'] = 1
df_ext = df_ext.pivot_table(index ='id', columns='Role', values = 'dummy', aggfunc=np.sum )

# we restore the column ID by restting the index
df_ext = df_ext.reset_index()

In [161]:
# we write the externals table to a csv file
df_ext.to_csv( '../data/'+ input_type + '_externals_raw.csv',index = False)


In [26]:
# this should be 0
df_p.loc[:,'Person.ID'].isnull().sum()

0

In [27]:
# we can now continue with the person table
# we make all categorial variables to strings
# numerics are first converted to integers, otherwise we will have .0 at the end
df_p[['Person.ID']] = df_p[['Person.ID']].astype(int).astype(str)
df_p[['Dept.No.','Faculty.No.']] = df_p[['Dept.No.','Faculty.No.']].astype(str)

In [28]:
# next columns to work on is the years spend in the uni
# we rename the column
df_p = df_p.rename(columns={'No..of.Years.in.Uni.at.Time.of.Grant': 'years_in_uni'})
# we use a dict to remap the values
dict_years_in_uni = {'Less than 0': 0.,
'>=0 to 5': 1.,
'>5 to 10': 2.,
'>10 to 15': 3.,
'more than 15': 4.}
df_p = df_p.replace({'years_in_uni' : dict_years_in_uni})
df_p.tail()

,id,Person.ID,Role,Year.of.Birth,Country.of.Birth,Home.Language,Dept.No.,Faculty.No.,With.PHD,years_in_uni,Number.of.Successful.Grant,Number.of.Unsuccessful.Grant,A.,A,B,C
5815,5816,69732,CHIEF_INVESTIGATOR,1950.0,Australia,NaN,2153.0,19.0,Yes,4.0,4.0,1.0,1.0,39.0,2.0,0.0
4126,4127,74722,CHIEF_INVESTIGATOR,1960.0,Australia,NaN,3048.0,31.0,Yes,3.0,1.0,0.0,13.0,5.0,10.0,3.0
4283,4284,74722,CHIEF_INVESTIGATOR,1960.0,Australia,NaN,3048.0,31.0,Yes,3.0,1.0,0.0,18.0,6.0,11.0,3.0
4126,4127,79977,CHIEF_INVESTIGATOR,1935.0,Australia,NaN,3048.0,31.0,NaN,4.0,1.0,0.0,10.0,3.0,2.0,NaN
4283,4284,79977,CHIEF_INVESTIGATOR,1935.0,Australia,NaN,3048.0,31.0,NaN,4.0,1.0,0.0,13.0,3.0,2.0,NaN


In [29]:
# we split it in two dataframes: static and dynamic
static_columns = ['Person.ID','Year.of.Birth','Country.of.Birth','Home.Language']
df_p_static = df_p.loc[:,static_columns]
df_p_static.to_csv(  '../data/'+ input_type + '_person_static_raw.csv',index = False)



In [30]:
# in the dynamic df comes ecervything that is not in the static + the key
df_p_dyn = df_p.loc[:,( (~(df_p.columns.isin(static_columns))) | df_p.columns.isin(['id','Person.ID']))]
# we only remove the Role, since we will have a seperate table for this
df_p_dyn = df_p_dyn.loc[:,~df_p_dyn.columns.isin(['Role']) ]

#df_p_before_join = df_p_dyn.loc[:,'id'].size

df_p_dyn  = pd.merge( df.loc[:,['id','date']], df_p_dyn, on = 'id')
#print(df_p_dyn.loc[:,'id'].size == df_p_before_join)

df_p_dyn = df_p_dyn.loc[:,~df_p_dyn.columns.isin(['id']) ]

df_p_dyn.to_csv(  '../data/'+ input_type + '_person_dyn_raw.csv',index = False)

In [31]:
#we extract the roles
roles = df_p.loc[:,['id','Person.ID','Roles']]

In [32]:
roles.to_csv(  '../data/'+ input_type + '_roles_raw.csv',index = False)

In [13]:
df_p_dyn.loc[df_p_dyn.loc[:,'Dept.No.'] == 'nan',:] = np.nan
df_p_dyn.loc[df_p_dyn.loc[:,'Dept.No.'] == 'nan',:]

,date,Person.ID,Dept.No.,Faculty.No.,With.PHD,No..of.Years.in.Uni.at.Time.of.Grant,Number.of.Successful.Grant,Number.of.Unsuccessful.Grant,A.,A,B,C
17,19/11/05,89462,nan,nan,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0
45,19/11/05,3067,nan,nan,NaN,NaN,0.0,0.0,3.0,3.0,1.0,0.0
46,19/11/05,51027,nan,nan,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
56,19/11/05,537,nan,25.0,Yes,>10 to 15,2.0,0.0,9.0,9.0,13.0,2.0
71,19/11/05,78817,nan,25.0,Yes,more than 15,2.0,0.0,4.0,8.0,6.0,2.0
88,19/11/05,65227,nan,nan,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
90,19/11/05,46237,nan,nan,NaN,NaN,0.0,0.0,0.0,1.0,2.0,1.0
98,19/11/05,41202,nan,nan,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
164,19/11/05,17512,nan,nan,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
178,19/11/05,64782,nan,nan,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0


In [220]:
df_p_dyn.groupby('Person.ID').size().sort_values()

Person.ID
99877      1
7037       1
30582      1
30612      1
3062       1
30632      1
703262     1
70192      1
30777      1
30827      1
700267     1
700037     1
30987      1
30997      1
704062     1
31012      1
699612     1
31237      1
69952      1
313557     1
699007     1
314067     1
69887      1
314557     1
697842     1
314962     1
31512      1
315512     1
69782      1
69977      1
          ..
71867     20
1757      20
76352     21
66887     21
79032     21
78817     22
9067      22
34737     22
69032     22
734322    23
46132     23
84622     24
28607     24
498427    24
73757     24
16137     25
69812     26
72037     28
82567     28
67647     30
67037     31
12957     32
74162     32
36252     33
17342     33
85862     35
67842     37
85642     39
79192     57
1797      72
Length: 2498, dtype: int64

In [55]:
df_p.loc[df_p.Role == 'STUD_CHIEF_INVESTIGATOR','Person.ID'].unique()

array([ nan])

In [52]:
df_p.loc[ df_p.loc[:,'Person.ID'] == 489582,:]

,id,Person.ID,Role,Year.of.Birth,Country.of.Birth,Home.Language,Dept.No.,Faculty.No.,With.PHD,No..of.Years.in.Uni.at.Time.of.Grant,Number.of.Successful.Grant,Number.of.Unsuccessful.Grant,A.,A,B,C
69,70,489582.0,CHIEF_INVESTIGATOR,1975.0,Great Britain,NaN,3048.0,31.0,Yes,>=0 to 5,0.0,1.0,11.0,1.0,2.0,0.0
2427,2428,489582.0,CHIEF_INVESTIGATOR,1975.0,Great Britain,NaN,3048.0,31.0,Yes,>=0 to 5,1.0,1.0,12.0,1.0,3.0,1.0
3173,3174,489582.0,CHIEF_INVESTIGATOR,1975.0,Great Britain,NaN,3048.0,31.0,Yes,>=0 to 5,2.0,1.0,12.0,1.0,3.0,1.0
4221,4222,489582.0,CHIEF_INVESTIGATOR,1975.0,Great Britain,NaN,3048.0,31.0,Yes,>=0 to 5,2.0,1.0,13.0,1.0,3.0,1.0
4828,4829,489582.0,CHIEF_INVESTIGATOR,1975.0,Great Britain,NaN,3048.0,31.0,Yes,>=0 to 5,2.0,2.0,13.0,1.0,3.0,1.0
5493,5494,489582.0,CHIEF_INVESTIGATOR,1975.0,Great Britain,NaN,3048.0,31.0,Yes,>=0 to 5,2.0,2.0,13.0,1.0,3.0,1.0
3154,3155,489582.0,CHIEF_INVESTIGATOR,1975.0,Great Britain,NaN,3048.0,31.0,Yes,>=0 to 5,1.0,1.0,0.0,0.0,0.0,0.0
3972,3973,489582.0,CHIEF_INVESTIGATOR,1975.0,Great Britain,NaN,3048.0,31.0,Yes,>=0 to 5,2.0,1.0,0.0,0.0,0.0,0.0
6183,6184,489582.0,CHIEF_INVESTIGATOR,1975.0,Great Britain,NaN,3048.0,31.0,Yes,>=0 to 5,2.0,2.0,0.0,0.0,0.0,0.0
5725,5726,489582.0,CHIEF_INVESTIGATOR,1975.0,Great Britain,NaN,3048.0,31.0,Yes,>=0 to 5,2.0,2.0,13.0,1.0,3.0,1.0


In [141]:
df.date

0       2005-08-11
1       2005-11-11
2       2005-11-14
3       2005-11-15
4       2005-11-16
5       2005-11-19
6       2005-11-19
7       2005-11-19
8       2005-11-19
9       2005-11-19
10      2005-11-19
11      2005-11-19
12      2005-11-19
13      2005-11-19
14      2005-11-19
15      2005-11-19
16      2005-11-19
17      2005-11-19
18      2005-11-19
19      2005-11-19
20      2005-11-19
21      2005-11-19
22      2005-11-19
23      2005-11-19
24      2005-11-19
25      2005-11-19
26      2005-11-19
27      2005-11-19
28      2005-11-19
29      2005-11-19
           ...    
6603    2007-10-12
6604    2007-11-12
6605    2007-11-12
6606    2007-12-12
6607    2007-12-12
6608    2007-12-13
6609    2007-12-13
6610    2007-12-13
6611    2007-12-14
6612    2007-12-14
6613    2007-12-14
6614    2007-12-14
6615    2007-12-14
6616    2007-12-17
6617    2007-12-17
6618    2007-12-17
6619    2007-12-17
6620    2007-12-17
6621    2007-12-18
6622    2007-12-18
6623    2007-12-18
6624    2007

In [171]:
df_p.loc[:,'Role'].unique()

array(['CHIEF_INVESTIGATOR', 'PRINCIPAL_SUPERVISOR',
       'DELEGATED_RESEARCHER', 'HONVISIT', nan], dtype=object)

In [29]:
df = pd.read_csv('../data/' + input_type + '_ml.csv', low_memory=False)

In [187]:
df.shape[1]

70

In [6]:
pd.set_option('display.max_columns', None)

In [184]:
def extract(df_train, df_test):

    Y_train = df_train['granted']
    Y_test = df_test['granted']
    X_train = df_train.drop(['granted', 'id'],axis=1)
    X_test = df_test.drop(['granted', 'id'],axis=1)

    return X_train, y_train, X_test, y_test

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout



model = Sequential()
model.add(Dense(64, input_dim=20, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          epochs=20,
          batch_size=128)
score = model.evaluate(x_test, y_test, batch_size=128)
